In [25]:
import pandas as pd

In [26]:
# Load the data
pd_excel_file = pd.ExcelFile("../data/landing/historical_data.xlsx")

In [27]:
processed_data = []

In [28]:
def parse_sheet(df_sheet):
    # sheet_0 = pd_excel_file.parse(0, header=None)
    df_sheet[0] = df_sheet[0].ffill()
    
    # forward fill the first row
    df = df_sheet.ffill(axis=1)
    
    print(df.head())
    return df

In [29]:
for sheet in range(len(pd_excel_file.sheet_names)):
    df_sheet = pd_excel_file.parse(sheet, header=None)
    break

In [30]:
# Concatenate the processed data
df_sheet.head()
df_sheet[0] = df_sheet[0].ffill()
df_sheet = df_sheet.ffill(axis=1)
# Combine the first two rows
df_sheet[0] = df_sheet[1] + " " + df_sheet[2]
df_sheet.head()

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,Moving annual rent by suburb Moving annual ren...,Moving annual rent by suburb,Lease commenced in year ending,Lease commenced in year ending,Lease commenced in year ending,Lease commenced in year ending,Lease commenced in year ending,Lease commenced in year ending,Lease commenced in year ending,Lease commenced in year ending,...,Lease commenced in year ending,Lease commenced in year ending,Lease commenced in year ending,Lease commenced in year ending,Lease commenced in year ending,Lease commenced in year ending,Lease commenced in year ending,Lease commenced in year ending,Lease commenced in year ending,Lease commenced in year ending
1,1 bedroom flat 1 bedroom flat,1 bedroom flat,Mar 2000,Mar 2000,Jun 2000,Jun 2000,Sep 2000,Sep 2000,Dec 2000,Dec 2000,...,Mar 2022,Mar 2022,Jun 2022,Jun 2022,Sep 2022,Sep 2022,Dec 2022,Dec 2022,Mar 2023,Mar 2023
2,1 bedroom flat 1 bedroom flat,1 bedroom flat,Count,Median,Count,Median,Count,Median,Count,Median,...,Count,Median,Count,Median,Count,Median,Count,Median,Count,Median
3,Inner Melbourne Albert Park-Middle Park-West S...,Albert Park-Middle Park-West St Kilda,352,165,347,165,378,170,369,175,...,269,315,262,325,277,340,260,350,262,360
4,Inner Melbourne Armadale,Armadale,210,150,212,150,213,155,213,160,...,223,310,233,320,237,340,238,350,204,360
